# Time based reports

A single function that:

- Slices your given date range into daily/weekly/monthly UTC intervals (localized to your timezone),  
- Fetches aggregated tag values (mean, max, sum, etc.) from TrendMiner in bulk,  
- Merges all results into one DataFrame and applies any custom pandas‐style expressions you define.  

**Inputs:**  
`start_date`, `end_date`, `frequency` (“daily”/“weekly”/“monthly”), `target_timezone`,  
`tags_aggs` (column→(tagID, agg)), `tags_compound_calculations` (newCol→expression)

**Output:**  
A sorted `pandas.DataFrame` with one row per interval and columns for each tag and compound metric.   

In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz

def compute_timebased_report(
    start_date: datetime,
    end_date: datetime,
    frequency: str,
    target_timezone: str,
    tags_aggs: dict,
) -> pd.DataFrame:
    """
    Generate a time‐based report over a given interval and frequency,
    pulling raw tag‐aggregation data from TrendMiner and applying
    any compound calculations.

    Parameters
    ----------
    start_date : datetime
        Start of the overall period (naive or tz‐aware).
    end_date : datetime
        End of the overall period (naive or tz‐aware).
    frequency : {'daily','weekly','monthly'}
        How to slice the period.
    target_timezone : str
        E.g. 'Europe/Brussels' — used to localize each slice.
    tags_aggs : dict
        e.g. {'TagA': ('BA:TEMP.1','mean'), ...}

    Returns
    -------
    pd.DataFrame
        Columns = startDate, endDate, each tag agg, each compound calc.
    """

    # ——— ENVIRONMENT & AUTH ———
    token = os.environ["KERNEL_USER_TOKEN"]
    server_url = os.environ["KERNEL_SERVER_URL"]
    auth_header = {"Authorization": f"Bearer {token}"}

    # ——— NESTED HELPERS ———

    def _utc_str(dt: datetime) -> str:
        # convert any tz‐aware dt to UTC and format as ISOZ
        utc = dt.astimezone(pytz.UTC)
        return utc.strftime("%Y-%m-%dT%H:%M:%S.000Z")

    def get_day_ranges():
        tz = pytz.timezone(target_timezone)
        periods = []
        cur = start_date.astimezone(tz).replace(hour=0, minute=0, second=0, microsecond=0)
        end_loc = end_date.astimezone(tz)
        idx = 1
        while cur < end_loc:
            nxt = cur + timedelta(days=1)
            periods.append({
                "startDate": _utc_str(cur),
                "endDate":   _utc_str(nxt),
                "key":       f"key{idx}"
            })
            cur = nxt
            idx += 1
        return periods

    def get_week_ranges():
        tz = pytz.timezone(target_timezone)
        periods = []
        cur = start_date.astimezone(tz)
        end_loc = end_date.astimezone(tz)
        idx = 1
        while cur <= end_loc:
            week_start = (cur - timedelta(days=cur.weekday())).replace(hour=0, minute=0, second=0, microsecond=0)
            next_week = week_start + timedelta(days=7)
            periods.append({
                "startDate": _utc_str(week_start),
                "endDate":   _utc_str(next_week),
                "key":       f"key{idx}"
            })
            cur = next_week
            idx += 1
        return periods

    def get_month_ranges():
        tz = pytz.timezone(target_timezone)
        periods = []
        # align to first of month
        cur = start_date.astimezone(tz)
        cur = cur.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
        end_loc = end_date.astimezone(tz)
        idx = 1
        while cur < end_loc:
            nxt = (cur + relativedelta(months=1)).replace(day=1)
            periods.append({
                "startDate": _utc_str(cur),
                "endDate":   _utc_str(nxt),
                "key":       f"key{idx}"
            })
            cur = nxt
            idx += 1
        return periods

    def get_tag_details(tag_id: str) -> dict:
        params = {"tagName": tag_id}
        r = requests.get(f"{server_url}/hps/api/tags/details",
                         params=params,
                         headers=auth_header)
        r.raise_for_status()
        return r.json()

    def get_calculations(tag_id: str, interpolation: str, agg: str, periods: list) -> list:
        body = {
            "queries": {
                "tag": {
                    "id":tag_id,
                    "shift":0,
                    "interpolationType": interpolation
                },
                "timePeriods": periods
            },
            "parameters": {"calculationType": agg}
        }
        r = requests.post(f"{server_url}/compute/newCalculation",
                          json=body,
                          headers=auth_header)
        r.raise_for_status()
        return r.json()

    # ——— BUILD THE TIME SLICES ———
    if frequency == "daily":
        slices = get_day_ranges()
    elif frequency == "weekly":
        slices = get_week_ranges()
    elif frequency == "monthly":
        slices = get_month_ranges()
    else:
        raise ValueError("frequency must be 'daily', 'weekly' or 'monthly'")

    # ——— PULL & AGGREGATE PER TAG ———
    report = pd.DataFrame()
    for col_name, (tag_id, agg_func) in tags_aggs.items():
        # 1) find how TrendMiner wants to interpolate
        details = get_tag_details(tag_id)
        interp = details.get("interpolationType", "Linear")
        # 2) pull the values
        raw = get_calculations(tag_id, interp, agg_func, slices)
        df  = pd.DataFrame(raw).rename(columns={"value": col_name})
        # 3) merge into master
        if report.empty:
            report = df
        else:
            report = report.merge(
                df,
                on=["startDate", "endDate", "key"],
                how="outer"
            )
    # ——— FINAL CLEANUP ———
    report["startDate"] = pd.to_datetime(report["startDate"])
    report["endDate"]   = pd.to_datetime(report["endDate"])
    if "key" in report.columns:
        report = report.drop(columns=["key"])
    report = report.sort_values("startDate").reset_index(drop=True)

    return report

In [ ]:
from datetime import datetime
# define your time‐window and tags
start = datetime(2025, 1, 1)
end   = datetime(2025, 5, 1)
tags_aggs = {
    "AvgTemp": ("[CS]BA:TEMP.1", "mean"),
}

df = compute_timebased_report(
    start_date=start,
    end_date=end,
    frequency="monthly",
    target_timezone="Europe/Brussels",
    tags_aggs=tags_aggs,
)
df